# Predicting Cryptocurrency Price With Tensorflow and Keras
黃功詳 Steeve Huang

source 
https://medium.com/@huangkh19951228/predicting-cryptocurrency-price-with-tensorflow-and-keras-e1674b0dc58a


In [1]:
import json
import numpy as np
import os
import pandas as pd



If the data file  'data/bitcoin2015to2017.csv' exits , no need to run to is download.

In [ ]:
import urllib2

# connect to poloniex's API
url = 'https://poloniex.com/public?command=returnChartData&currencyPair=USDT_BTC&start=1356998100&end=9999999999&period=300'

# parse json returned from the API to Pandas DF
openUrl = urllib2.urlopen(url)
r = openUrl.read()
openUrl.close()
d = json.loads(r.decode())
df = pd.DataFrame(d)


In [8]:
df.columns

Index([u'close', u'date', u'high', u'low', u'open', u'quoteVolume', u'volume',
       u'weightedAverage'],
      dtype='object')

In [9]:
original_columns=[u'close', u'date', u'high', u'low', u'open']
new_columns = ['Close','Timestamp','High','Low','Open']
df = df.loc[:,original_columns]
df.columns = new_columns


In [ ]:
df.to_csv('data/bitcoin2015to2017.csv',index=None)

In [1]:
import numpy as np
import pandas as pd
 
class PastSampler:
    '''
    Forms training samples for predicting future values from past value
    '''
     
    def __init__(self, N, K, sliding_window = True):
        '''
        Predict K future sample using N previous samples
        '''
        self.K = K
        self.N = N
        self.sliding_window = sliding_window
 
    def transform(self, A):
        M = self.N + self.K     #Number of samples per row (sample + target)
        #indexes
        if self.sliding_window:
            I = np.arange(M) + np.arange(A.shape[0] - M + 1).reshape(-1, 1)
        else:
            if A.shape[0]%M == 0:
                I = np.arange(M)+np.arange(0,A.shape[0],M).reshape(-1,1)
                
            else:
                I = np.arange(M)+np.arange(0,A.shape[0] -M,M).reshape(-1,1)
            
        B = A[I].reshape(-1, M * A.shape[1], A.shape[2])
        ci = self.N * A.shape[1]    #Number of features per sample
        return B[:, :ci], B[:, ci:] #Sample matrix, Target matrix


In [5]:
pd.read_csv('data/bitcoin2015to2017.csv').head()

,Close,Timestamp,High,Low,Open
0,225.0,1424373000,0.33,225.0,0.33
1,225.0,1424373300,225.00,225.0,225.00
2,225.0,1424373600,225.00,225.0,225.00
3,225.0,1424373900,225.00,225.0,225.00
4,225.0,1424374200,225.00,225.0,225.00


In [2]:
#data file path
dfp = 'data/bitcoin2015to2017.csv'

#Columns of price data to use
columns = ['Close']
df = pd.read_csv(dfp)
time_stamps = df['Timestamp']
df = df.loc[:,columns]
original_df = pd.read_csv(dfp).loc[:,columns]

In [3]:
df.head()

,Close
0,225.0
1,225.0
2,225.0
3,225.0
4,225.0


In [6]:
original_df.head()

,Close
0,225.0
1,225.0
2,225.0
3,225.0
4,225.0


In [6]:
df['Close'].values.reshape(-1,1)

array([[   225.       ],
       [   225.       ],
       [   225.       ],
       ..., 
       [ 15829.9999984],
       [ 15796.2119108],
       [ 15796.2119106]])

In [7]:
file_name='bitcoin2015to2017_close.h5'

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# normalization
for c in columns:
    df[c] = scaler.fit_transform(df[c].values.reshape(-1,1))


In [8]:
df.head()

,Close
0,0.006364
1,0.006364
2,0.006364
3,0.006364
4,0.006364


In [16]:
np.array(df) #[:] #[:,None,None,None]

array([[ 0.00636438],
       [ 0.00636438],
       [ 0.00636438],
       ..., 
       [ 0.79458776],
       [ 0.79288109],
       [ 0.79288109]])

In [ ]:
    
#Features are input sample dimensions(channels)
A = np.array(df)[:,None,:]
original_A = np.array(original_df)[:,None,:]
time_stamps = np.array(time_stamps)[:,None,None]


In [8]:

#Make samples of temporal sequences of pricing data (channel)
NPS, NFS = 256, 16         #Number of past and future samples
ps = PastSampler(NPS, NFS, sliding_window=False)
B, Y = ps.transform(A)
input_times, output_times = ps.transform(time_stamps)
original_B, original_Y = ps.transform(original_A)

In [9]:
import h5py
with h5py.File(file_name, 'w') as f:
    f.create_dataset("inputs", data = B)
    f.create_dataset('outputs', data = Y)
    f.create_dataset("input_times", data = input_times)
    f.create_dataset('output_times', data = output_times)
    f.create_dataset("original_datas", data=np.array(original_df))
    f.create_dataset('original_inputs',data=original_B)
    f.create_dataset('original_outputs',data=original_Y)

# CNN

In [10]:
import pandas as pd
import numpy as numpy
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv1D, MaxPooling1D, LeakyReLU, PReLU
from keras.utils import np_utils
from keras.callbacks import CSVLogger, ModelCheckpoint
import h5py
import os
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

'''
# Make the program use only one GPU
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
'''

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))


with h5py.File(''.join(['bitcoin2015to2017_close.h5']), 'r') as hf:
    datas = hf['inputs'].value
    labels = hf['outputs'].value


output_file_name='bitcoin2015to2017_close_CNN_2_relu'

step_size = datas.shape[1]
batch_size= 8
nb_features = datas.shape[2]

epochs = 100

#split training validation
training_size = int(0.8* datas.shape[0])
training_datas = datas[:training_size,:]
training_labels = labels[:training_size,:]
validation_datas = datas[training_size:,:]
validation_labels = labels[training_size:,:]
#build model

# 2 layers
model = Sequential()

'''
model.add(Conv1D(activation='relu', input_shape=(step_size, nb_features), strides=3, filters=8, kernel_size=20))
model.add(Dropout(0.5))
model.add(Conv1D( strides=4, filters=nb_features, kernel_size=16))
'''

# 3 Layers
model.add(Conv1D(activation='relu', 
                 input_shape=(step_size, nb_features), 
                 strides=3, filters=8, kernel_size=8))
#model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Conv1D(activation='relu', strides=2, filters=8, kernel_size=8))
#model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Conv1D( strides=2, filters=nb_features, kernel_size=8))
'''
# 4 layers
model.add(Conv1D(activation='relu', input_shape=(step_size, nb_features), strides=2, filters=8, kernel_size=2))
#model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Conv1D(activation='relu', strides=2, filters=8, kernel_size=2))
#model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Conv1D(activation='relu', strides=2, filters=8, kernel_size=2))
#model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Conv1D( strides=2, filters=nb_features, kernel_size=2))
'''

ModuleNotFoundError: No module named 'keras'

In [25]:
model.compile(loss='mse', optimizer='adam')
model.fit(training_datas, training_labels,verbose=1, 
          batch_size=batch_size,
          validation_data=(validation_datas,validation_labels), 
          epochs = epochs, 
          callbacks=[CSVLogger(output_file_name+'.csv', append=True),
                     ModelCheckpoint('weights/'+output_file_name+'-{epoch:02d}-{val_loss:.5f}.hdf5', 
                                     monitor='val_loss', verbose=1,mode='min')])

Train on 892 samples, validate on 223 samples
Epoch 1/100
892/892 [==============================] - 0s - loss: 2.0824e-04 - val_loss: 0.0016
Epoch 2/100
892/892 [==============================] - 0s - loss: 7.1827e-05 - val_loss: 0.0026
Epoch 3/100
892/892 [==============================] - 0s - loss: 6.1782e-05 - val_loss: 4.8151e-04
Epoch 4/100
892/892 [==============================] - 0s - loss: 4.8609e-05 - val_loss: 3.9420e-04
Epoch 5/100
892/892 [==============================] - 0s - loss: 4.4719e-05 - val_loss: 4.8149e-04
Epoch 6/100
892/892 [==============================] - 0s - loss: 4.0770e-05 - val_loss: 9.6418e-04
Epoch 7/100
892/892 [==============================] - 0s - loss: 4.4148e-05 - val_loss: 0.0016
Epoch 8/100
892/892 [==============================] - 0s - loss: 4.1305e-05 - val_loss: 7.7556e-04
Epoch 9/100
892/892 [==============================] - 0s - loss: 4.1489e-05 - val_loss: 3.2940e-04
Epoch 10/100
892/892 [==============================] - 0s - loss:

# LSTM 

In [27]:
import pandas as pd
import numpy as numpy
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,Reshape
from keras.layers import Conv1D, MaxPooling1D
from keras.utils import np_utils
from keras.layers import LSTM, LeakyReLU
from keras.callbacks import CSVLogger, ModelCheckpoint
import h5py
import os
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session


'''
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
'''

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

with h5py.File(''.join(['bitcoin2015to2017_close.h5']), 'r') as hf:
    datas = hf['inputs'].value
    labels = hf['outputs'].value




step_size = datas.shape[1]
units= 50
second_units = 30
batch_size = 8
nb_features = datas.shape[2]
epochs = 100
output_size=16
output_file_name='bitcoin2015to2017_close_LSTM_1_tanh_leaky_'
#split training validation
training_size = int(0.8* datas.shape[0])
training_datas = datas[:training_size,:]
training_labels = labels[:training_size,:,0]
validation_datas = datas[training_size:,:]
validation_labels = labels[training_size:,:,0]


#build model
model = Sequential()
model.add(LSTM(units=units,activation='tanh', 
               input_shape=(step_size,nb_features),return_sequences=False))
model.add(Dropout(0.8))
model.add(Dense(output_size))
model.add(LeakyReLU())
model.compile(loss='mse', optimizer='adam')
model.fit(training_datas, training_labels, 
          batch_size=batch_size,validation_data=(validation_datas,validation_labels), epochs = epochs, callbacks=[CSVLogger(output_file_name+'.csv', append=True),ModelCheckpoint('weights/'+output_file_name+'-{epoch:02d}-{val_loss:.5f}.hdf5', monitor='val_loss', verbose=1,mode='min')])

Train on 892 samples, validate on 223 samples
Epoch 1/100
892/892 [==============================] - 14s - loss: 4.0536e-04 - val_loss: 0.0284
Epoch 2/100
892/892 [==============================] - 14s - loss: 1.9366e-04 - val_loss: 0.0138
Epoch 3/100
892/892 [==============================] - 20s - loss: 1.3637e-04 - val_loss: 0.0078
Epoch 4/100
892/892 [==============================] - 13s - loss: 1.1355e-04 - val_loss: 0.0107
Epoch 5/100
892/892 [==============================] - 13s - loss: 1.1288e-04 - val_loss: 0.0036
Epoch 6/100
892/892 [==============================] - 13s - loss: 1.0942e-04 - val_loss: 8.4948e-04
Epoch 7/100
892/892 [==============================] - 21s - loss: 8.0963e-05 - val_loss: 0.0066
Epoch 8/100
892/892 [==============================] - 13s - loss: 9.3582e-05 - val_loss: 0.0084
Epoch 9/100
892/892 [==============================] - 14s - loss: 8.9833e-05 - val_loss: 0.0093
Epoch 10/100
892/892 [==============================] - 19s - loss: 7.3528e-0

# GRU

In [12]:
import pandas as pd
import numpy as numpy
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,Reshape
from keras.layers import Conv1D, MaxPooling1D, LeakyReLU
from keras.utils import np_utils
from keras.layers import GRU
#from keras.layers import CuDNNGRU
from keras.callbacks import CSVLogger, ModelCheckpoint
import h5py
import os
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session


'''
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
'''

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

with h5py.File(''.join(['bitcoin2015to2017_close.h5']), 'r') as hf:
    datas = hf['inputs'].value
    labels = hf['outputs'].value


output_file_name='bitcoin2015to2017_close_GRU_1_tanh_relu_'

step_size = datas.shape[1]
units= 50
batch_size = 8
nb_features = datas.shape[2]
epochs = 100
output_size=16
#split training validation
training_size = int(0.8* datas.shape[0])
training_datas = datas[:training_size,:]
training_labels = labels[:training_size,:,0]
validation_datas = datas[training_size:,:]
validation_labels = labels[training_size:,:,0]

#build model
model = Sequential()
model.add(GRU(units=units, input_shape=(step_size,nb_features),return_sequences=False))
model.add(Activation('tanh'))
model.add(Dropout(0.2))
model.add(Dense(output_size))
model.add(Activation('relu'))
model.compile(loss='mse', optimizer='adam')
model.fit(training_datas, training_labels, batch_size=batch_size,validation_data=(validation_datas,validation_labels), epochs = epochs, callbacks=[CSVLogger(output_file_name+'.csv', append=True),ModelCheckpoint('weights/'+output_file_name+'-{epoch:02d}-{val_loss:.5f}.hdf5', monitor='val_loss', verbose=1,mode='min')])

Train on 886 samples, validate on 222 samples
Epoch 1/100
886/886 [==============================] - 11s - loss: 2.3968e-04 - val_loss: 0.0013
Epoch 2/100
886/886 [==============================] - 11s - loss: 2.5543e-05 - val_loss: 6.3723e-04
Epoch 3/100
886/886 [==============================] - 11s - loss: 1.5321e-05 - val_loss: 3.7577e-04
Epoch 4/100
886/886 [==============================] - 11s - loss: 1.5172e-05 - val_loss: 8.1988e-05
Epoch 5/100
886/886 [==============================] - 11s - loss: 1.3094e-05 - val_loss: 9.9603e-05
Epoch 6/100
886/886 [==============================] - 11s - loss: 1.0659e-05 - val_loss: 2.6188e-04
Epoch 7/100
886/886 [==============================] - 11s - loss: 1.1271e-05 - val_loss: 2.9588e-04
Epoch 8/100
886/886 [==============================] - 11s - loss: 1.0345e-05 - val_loss: 7.2698e-05
Epoch 9/100
886/886 [==============================] - 11s - loss: 9.0200e-06 - val_loss: 1.0582e-04
Epoch 10/100
886/886 [===========================